In [5]:
import pandas as pd
df = pd.read_csv('train_essays_3.0.csv')

In [6]:
import numpy as np
from tqdm import tqdm 


train_df = pd.read_csv('train_essays_3.0.csv')
print(f"Train essays dataframe has shape: {train_df.shape}")

'''
    Stratified K Fold
'''

real_indice = train_df[train_df['generated']==0].index
fake_indice = train_df[train_df['generated']==1].index
real_choice_valid = np.random.choice(real_indice, 1000, replace=False)
fake_choice_valid = np.random.choice(fake_indice, 1000, replace=False)
train_df['fold'] = 1
train_df.loc[real_choice_valid, 'fold'] = 0
train_df.loc[fake_choice_valid, 'fold'] = 0
# skf = StratifiedKFold(n_splits=5)

# X = train_df.loc[:, train_df.columns != "generated"]
# y = train_df.loc[:, train_df.columns == "generated"]



# for i, (train_index, valid_index) in enumerate(skf.split(X, y)):
#     train_df.loc[valid_index, "fold"] = i
    
print(train_df.groupby("fold")["generated"].value_counts())
train_df.head()

oof_df = pd.DataFrame()

Train essays dataframe has shape: (32722, 4)
fold  generated
0     0             1000
      1             1000
1     0            26371
      1             4351
Name: count, dtype: int64


In [7]:
folds = train_df
fold = 0


In [8]:
from torch.utils.data import DataLoader, Dataset
import torch
def prepare_input(cfg, text, tokenizer):
    """
    This function tokenizes the input text with the configured padding and truncation. Then,
    returns the input dictionary, which contains the following keys: "input_ids",
    "token_type_ids" and "attention_mask". Each value is a torch.tensor.
    :param cfg: configuration class with a TOKENIZER attribute.
    :param text: a numpy array where each value is a text as string.
    :return inputs: python dictionary where values are torch tensors.
    """
    inputs = tokenizer.encode_plus(
        text, 
        return_tensors=None, 
        add_special_tokens=True, 
        max_length=cfg.MAX_LEN,
        padding='max_length', # TODO: check padding to max sequence in batch
        truncation=True
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long) # TODO: check dtypes
    return inputs

class CustomDataset(Dataset):
    def __init__(self, cfg, df, tokenizer):
        self.cfg = cfg
        self.texts = df['text'].values
        self.labels = df['generated'].values
        self.tokenizer = tokenizer
        self.text_ids = df['id'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        output = {}
        output["inputs"] = prepare_input(self.cfg, self.texts[item], self.tokenizer)
        output["labels"] = torch.tensor(self.labels[item], dtype=torch.float) # TODO: check dtypes
        output["ids"] = self.text_ids[item]
        return output


In [10]:
from transformers import AutoTokenizer
class config:
    APEX = True # Automatic Precision Enabled
    BATCH_SCHEDULER = True
    BATCH_SIZE_TRAIN = 32
    BATCH_SIZE_VALID = 16
    BETAS = (0.9, 0.999)
    DEBUG = False
    DECODER_LR = 2e-5
    ENCODER_LR = 2e-5
    EPOCHS = 5
    EPS = 1e-6
    FOLDS = 2
    GRADIENT_ACCUMULATION_STEPS = 1
    GRADIENT_CHECKPOINTING = True
    MAX_GRAD_NORM=1000
    MAX_LEN = 512
    MIN_LR = 1e-6
    MODEL = "microsoft/deberta-v3-base"
    NUM_CYCLES = 0.5
    NUM_WARMUP_STEPS = 0
    PRINT_FREQ = 20
    SCHEDULER = 'cosine' # ['linear', 'cosine']
    SEED = 27
    TRAIN = True
    TRAIN_FOLDS = [0, 1, 2, 3]
    WANDB = True
    WEIGHT_DECAY = 0.01


train_folds = folds[folds['fold'] != fold].reset_index(drop=True)
valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)

valid_labels = valid_folds['generated'].values

# ======== DATASETS ==========
label0_dataset = train_folds[train_folds['generated']==0]
label1_dataset = train_folds[train_folds['generated']==1]
dataset = pd.concat([label0_dataset,
                label1_dataset,
                label1_dataset,
                label1_dataset,
                label1_dataset,
                label1_dataset,
                ], 
                ignore_index=True)
train_folds = dataset
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
train_dataset = CustomDataset(config, train_folds, tokenizer)
valid_dataset = CustomDataset(config, valid_folds, tokenizer)

ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [11]:
import numpy as np

In [14]:
choice = np.random.choice(real_indice, 1000)

In [21]:
df.loc[choice, 'generated']

3495     0
7417     0
9294     0
18564    0
22473    0
        ..
20156    0
23124    0
681      0
3117     0
25264    0
Name: generated, Length: 1000, dtype: int64